# Lesson 5b: Recurrent Neural Networks: Categorization

Recurrent Neural Networks (RNNs) can be used in many different ways, such as classification, single-step prediction, and the generation of an entire sequence. 

* **Classification**: the input is a sequence, and the output is a single category - this is the focus of this assignment. (Alternatively, a sequence of categories could be generated, one for each partial sequence as it is processed).

* **Prediction**: the input is a sequence, and the output is a prediction for the next element in the sequence. You will explore this in lesson 5b.

* **Sequence Generation** (Seq-to-Seq): both the input and the output are entire sequences. For example, RNN-based language translation may take in an input sequence (of characters or word tokens) in English, and generate as output a sequence (of characters or word tokens) in French.

RNNs can be used to process inputs that occur naturally in time (such as an audio recording of speech or music represented as a stream of timestamped MIDI messages), but they can also be applied to material that has an order to it, even if it's not necessarly temporal in natures, such as written text (which can be read one character or one word at a time) or even written numbers or math equations (which can be read one digit or symbol at a time, from left to right, for instance.)  This is the problem we investigate today: looking at numbers such as "1423" as a sequence of digits ['1', '4', '2', '3'].

Our problem comes curtosy of Distinguished Professor Douglas R. Hofstadter of Indiana University, author of books such as _Gödel, Escher, Bach: an Eternal Golden Braid_. Hofstadter writes [private communication, shared with permission)]:

---

_Lately, I have been musing about the seeming power of deep neural nets.  They learn to recognize members of all sorts of categories, when those members (and non-members) are fed to them as patterns of symbols or of pixels.  So, how about the following challenges involving the natural numbers?_

* To recognize the even numbers, expressed in base 3.
     (Specifically, 0, 2, 11, 20, 22, 101, 110, 112,...)
* To recognize the multiples of 3, expressed in base 10.
* To recognize the multiples of 9, expressed in base 10.
* To recognize the multiples of 7, expressed in base 10.
* To recognize the multiples of 29, expressed in base 10.

_(I suppose that if a net can learn any particular one of the above list, it can learn all of them.  Just a guess...)_

 _Moving right along, how about the following somewhat harder challenges?_

* To recognize the correct integer additions, expressed either in base 2 or in base 10.  (For example, the string “12+29=41”.)
* To recognize the correct integer multiplications, expressed either in base 2 or in base 10.  (For example, the string “12x29=348”.)

_(The latter of this pair seems significantly harder than the former.)_
     
_And then, of course, the canonical challenge of this sort:_

* To recognize the prime numbers, expressed either in base 2 or in base 10.

_Each of the above challenges involves a number-theoretical category that can easily be described in purely syntactic terms (i.e., as a rule-based pattern of symbols).  It would be trivial to generate millions of examples of such categories mechanically, and then you just feed them to the neural net.  You can also feed the network lots of counterexamples -- marking them, of course, as non-members of the category.  Can a deep neural network learn any of these categories?  All of them?  Some of them?_

---

In this assignment, you will use an RNN to try to solve the divisibility-by-3 problem (the rest are challenges you might want to try in your free time!): 

* **"To recognize the multiples of 3, expressed in base 10."**  Specifically, you must:
    * Design an RNN that takes a sequence of digits as input. 
    * Represent digits in base 10 by using a categorical, one-hot encoding, with one node for each digit from 0 through 9.
    * Train the RNN to categorize a number as True if it is evenly divisible by 3, False otherwise.
    * Test the RNN on a set of previously-unseen numbers, including numbers that are 4 digits long, such as 2225 and 3333.
    * Acheive an accuracy of at least 95% on the test set (report the accuracy in the cell marked below).
    * Answer the questions at the end of this notebook.




- [One hot encoding](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/) 
- [Sequence Prediction](https://machinelearningmastery.com/sequence-prediction-problems-learning-lstm-recurrent-neural-networks/)
- [LSTM Tutorial Keras](https://adventuresinmachinelearning.com/keras-lstm-tutorial/)

# Setup
## Imports

- [Keras - Guide to the sequential model](https://keras.io/getting-started/sequential-model-guide/)
- [Input](https://keras.io/layers/core/)
- [GRU](https://keras.io/layers/recurrent/)
- [LSTM](https://keras.io/layers/recurrent/)
- [Dense](https://keras.io/layers/core/#Dense)
- [Masking](https://keras.io/layers/core/)
- [Dropout](https://keras.io/layers/core/)

- [Keras - fit](https://keras.io/models/sequential/)
- [Recurrent Neural Networks by Example in Python](https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470)
- [Sequence Classification with LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/)

Report your final accuracy on the validation dataset below.

## Examine model outputs

# Further Questions

1) What happens if you give a 5-digit number or a 6-digit number to the trained model, after training on 1-, 2-, 3-, and 4- digit numbers?

TODO

2) Pick another number from Hofstadter's list above, such as 9, 7, or 29. Train a model, and report the accuracy of your results. Did it work or not? Why or why not (your best guess)?

TODO

3) Record any other comments/insights from your model training process. What worked well? What caused trouble?

TODO

4) If you didn't have a training algorithm, how would you design a RNN-style system to recognize divisibility by 3?
Ignoring the details of the weights, what kind of state must be carried over from step to step as each digit is read in
successively?

TODO

5) BONUS (hard): Explain how the neural net you trained above works, with evidence from examining the node activations as the net runs. Does it do anything similar to what you would have designed as a human?

TODO

## From Jose:

In [1]:
%matplotlib inline
from keras.models import Model, Sequential, load_model
from keras.layers import Input, GRU, LSTM, Dense, Masking, Dropout, InputLayer, TimeDistributed
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import l1_l2
from keras.utils import to_categorical
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

/home/danka/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [30]:
# Set up params for dataset.
DIVISIBILITY_NUMBER = 3         # We want to test for divisilibity by 3.
TRAIN_TEST_SPLIT = 0.8          # Percentage of data in training set
NUM_EXAMPLES_PER_CLASS = 1000   # Generate the first 1000 multiples of 3 for training/testing
                                # Also generate 1000 non-multiples of 3.
NUM_CATEGORIES = 10             # 10 digits
MAX_DIGITS = 5                  # Number of digits allowed in input strings

# Neural net hyperparameters-- just an example. Adjust these as needed.
BATCH_SIZE = 32
NUM_LSTM_NODES = 10             
DROPOUT = 0.5
LEARING_RATE = 0.002
NUM_EPOCHS = 2500

# TODO: add/modify constants as needed

In [3]:
from random import sample

def generate_example_numbers(base_number=DIVISIBILITY_NUMBER, num_examples_per_class=NUM_EXAMPLES_PER_CLASS):
    """Return a tuple of two lists: (list_of_multiples, list_of_nonmultiples).
    
    For example, ([0, 3, 6, 9, 12, ...2997], [1, 4, 5, 8, 11, 13, 14,...,2999]).
    Each list contains num_examples_per_class elements.
    """
    #a = sample(range(0,num_examples_per_class * 60), num_examples_per_class * base_number * 2)
    # based on the density of the multiples within a set of integers
    #multiples = []
    #non_multiples = []
    #for i in a:
    #    if i % base_number == 0:
    #        multiples.append(i)
    #    else:
    #        non_multiples.append(i)
    
    #multiples = sample(multiples, num_examples_per_class)
    #non_multiples = sample(non_multiples, num_examples_per_class)
    
    #return (multiples, non_multiples)
    multiples = [n * base_number for n in range(num_examples_per_class)]
    non_multiples = [n * (base_number+1) for n in range(num_examples_per_class)]
    return (multiples, non_multiples)

In [4]:
def digit_to_vector(digit):
    """Given a digit from 0-9, return a numpy array representing the digit using a 1-hot encoding.
    keras.utils.to_categorical may be useful.
    """
    tmp = to_categorical(digit, dtype='int',num_classes=NUM_CATEGORIES)
    return tmp

In [5]:
def number_to_input_example(number, max_digits=MAX_DIGITS):
    """Given an integer number, return a numpy float array of 0.0s and 1.0s, of the correct shape to feed into the 
    neural net.
    
    For example, if you have a max of 5 digits then you should have a 2D numpy matrix: 5 rows (one for each
    sequence index), and 10 columns (1 for each digit).
    
    In order to train in "batch" mode, the RNN expects every example to have the same shape. So if you have a 2-digit
    number such as "42", you need to pad the example with a "padding" token somehow; for example, "???42", and then
    use keras.layers.Masking to ignore the leading digits. Or just pad with 0s, as in "00042". 
    keras.preprocessing.sequence.pad_sequences can help with this.
    """
    #a = pad_sequences([[int(i) for i in str(number)]], maxlen=max_digits, padding='pre')[0]
    #a = [digit_to_vector(i) for i in a]
    
    #return np.array(a)
    padded_number = pad_sequences([[int(k) for k in str(number)]], maxlen=max_digits)
    result = [np.zeros(10) for _ in range(max_digits)]
    for i, k in zip(range(max_digits), padded_number[0]):
      result[i][k] = 1
    
    return result

    

In [6]:
number_to_input_example(42, max_digits=MAX_DIGITS)

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])]

In [7]:
def generate_dataset(divisibility_number=DIVISIBILITY_NUMBER, 
                     train_test_split=TRAIN_TEST_SPLIT, 
                     num_examples_per_class=NUM_EXAMPLES_PER_CLASS):
    """Generate a dataset ready for training. Returns a list of tuples. Each tuple is of the form
    (input_array, label). The dataset should be shuffled either here or during the training process to
    mix divisile-by-DIVISIBILITY_NUMBER and not-divisible-by-DIVISIBILITY_NUMBER examples.
    The dataset should consist of NUM_EXAMPLES_PER_CLASS positive examples (e.g., 1000 examples of divisible-by-3), and
    also NUM_EXAMPLES_PER_CLASS negative examples (e.g., 1000 examples of not-divisible-by-3).
    """
    example_numbers = generate_example_numbers(divisibility_number, num_examples_per_class)
    divisible = example_numbers[0]
    non_divisible = example_numbers[1]
    np.random.shuffle(divisible)
    np.random.shuffle(non_divisible)
    
    result = []
    for k, m in zip(divisible, non_divisible):
        result.append((k, 1))
        result.append((m, 0))
        
    
    return result




In [23]:
# Build RNN model.
def build_model():
    model = Sequential()
    model.add(LSTM(NUM_LSTM_NODES, dropout=DROPOUT, recurrent_dropout=DROPOUT, batch_input_shape=(1,MAX_DIGITS,10), return_sequences=True))  # Use return_sequences=True for multiple hidden layers
    model.add(Dense(NUM_LSTM_NODES))
    model.add(LSTM(NUM_LSTM_NODES, dropout=DROPOUT, recurrent_dropout=DROPOUT, batch_input_shape=(1,MAX_DIGITS,10), return_sequences=False))  # Use return_sequences=True for multiple hidden layers
    model.add(Dense(1))  # Model should return 1 or 0 for divisible/not-divisible
    
    return model


# Print the model configuration.
model = build_model()

# Compile model
adam = Adam(lr=LEARING_RATE)   # Modify learning algorithm as needed
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

# Summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (1, 5, 10)                840       
_________________________________________________________________
dense_5 (Dense)              (1, 5, 10)                110       
_________________________________________________________________
lstm_5 (LSTM)                (1, 10)                   840       
_________________________________________________________________
dense_6 (Dense)              (1, 1)                    11        
Total params: 1,801
Trainable params: 1,801
Non-trainable params: 0
_________________________________________________________________


In [24]:
data = generate_dataset()

In [25]:
data[0]

(1539, 1)

In [26]:
data[0][0] / 3

513.0

In [27]:

X = np.array([number_to_input_example(d[0]) for d in data])
y = [d[1] for d in data]
n = len(X)

train_len = int(TRAIN_TEST_SPLIT * n)
train_inputs = X[:train_len]
train_labels = y[:train_len]
validation_inputs = X[train_len:]
validation_labels = y[train_len:]


# Train the model


In [28]:
print(sum(train_labels))
print(len(train_labels))
print(sum(validation_labels))
print(len(validation_labels))

800
1600
200
400


In [31]:
model.fit(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), 
          batch_size=1, epochs=NUM_EPOCHS, verbose=1)


Train on 1600 samples, validate on 400 samples
Epoch 1/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4088 - acc: 0.7400 - val_loss: 0.3380 - val_acc: 0.7475
Epoch 2/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.3854 - acc: 0.7194 - val_loss: 0.3364 - val_acc: 0.6800
Epoch 3/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.3839 - acc: 0.7006 - val_loss: 0.3346 - val_acc: 0.7000
Epoch 4/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4080 - acc: 0.7181 - val_loss: 0.3381 - val_acc: 0.7125
Epoch 5/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4045 - acc: 0.7262 - val_loss: 0.3417 - val_acc: 0.7375
Epoch 6/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4181 - acc: 0.7169 - val_loss: 0.3394 - val_acc: 0.7250
Epoch 7/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4064 - acc: 0.7062 - val_loss: 0.3396 - val_acc: 0.7000
E

Epoch 120/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3798 - acc: 0.7125 - val_loss: 0.3156 - val_acc: 0.6600
Epoch 121/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.4078 - acc: 0.7006 - val_loss: 0.3171 - val_acc: 0.6675
Epoch 122/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3700 - acc: 0.7200 - val_loss: 0.3158 - val_acc: 0.6125
Epoch 123/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3879 - acc: 0.6719 - val_loss: 0.3100 - val_acc: 0.6125
Epoch 124/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3721 - acc: 0.6725 - val_loss: 0.3129 - val_acc: 0.6125
Epoch 125/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3983 - acc: 0.6694 - val_loss: 0.3236 - val_acc: 0.6175
Epoch 126/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3935 - acc: 0.6937 - val_loss: 0.3190 - val_acc: 0.6150
Epoch 127/2500
1600/1600 [========

1600/1600 [==============================] - 7s 4ms/step - loss: 0.4086 - acc: 0.6906 - val_loss: 0.3108 - val_acc: 0.6575
Epoch 239/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.3925 - acc: 0.7144 - val_loss: 0.3105 - val_acc: 0.6675
Epoch 240/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4333 - acc: 0.7325 - val_loss: 0.3105 - val_acc: 0.6175
Epoch 241/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.3805 - acc: 0.7156 - val_loss: 0.3115 - val_acc: 0.6275
Epoch 242/2500
1600/1600 [==============================] - 9s 6ms/step - loss: 0.3613 - acc: 0.6937 - val_loss: 0.3239 - val_acc: 0.6800
Epoch 243/2500
1600/1600 [==============================] - 7s 5ms/step - loss: 0.3661 - acc: 0.6956 - val_loss: 0.3160 - val_acc: 0.6425
Epoch 244/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.3696 - acc: 0.6856 - val_loss: 0.3083 - val_acc: 0.6200
Epoch 245/2500
1600/1600 [=======================

1600/1600 [==============================] - 6s 4ms/step - loss: 0.4211 - acc: 0.6925 - val_loss: 0.3141 - val_acc: 0.6275
Epoch 357/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4472 - acc: 0.6512 - val_loss: 0.3170 - val_acc: 0.7075
Epoch 358/2500
1600/1600 [==============================] - 7s 5ms/step - loss: 0.3858 - acc: 0.6619 - val_loss: 0.3145 - val_acc: 0.6125
Epoch 359/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3985 - acc: 0.6613 - val_loss: 0.3109 - val_acc: 0.6825
Epoch 360/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.3953 - acc: 0.6763 - val_loss: 0.3090 - val_acc: 0.6150
Epoch 361/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.3942 - acc: 0.6669 - val_loss: 0.3087 - val_acc: 0.6200
Epoch 362/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4011 - acc: 0.6406 - val_loss: 0.3130 - val_acc: 0.6200
Epoch 363/2500
1600/1600 [=======================

1600/1600 [==============================] - 6s 4ms/step - loss: 0.3884 - acc: 0.6613 - val_loss: 0.3161 - val_acc: 0.6375
Epoch 475/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3778 - acc: 0.6644 - val_loss: 0.3164 - val_acc: 0.6225
Epoch 476/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3695 - acc: 0.6844 - val_loss: 0.3098 - val_acc: 0.6375
Epoch 477/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.3694 - acc: 0.6819 - val_loss: 0.3143 - val_acc: 0.6500
Epoch 478/2500
1600/1600 [==============================] - 9s 6ms/step - loss: 0.3621 - acc: 0.6706 - val_loss: 0.3072 - val_acc: 0.6125
Epoch 479/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4395 - acc: 0.6425 - val_loss: 0.3065 - val_acc: 0.6125
Epoch 480/2500
1600/1600 [==============================] - 9s 5ms/step - loss: 0.3880 - acc: 0.6581 - val_loss: 0.3080 - val_acc: 0.6125
Epoch 481/2500
1600/1600 [=======================

...

Epoch 2244/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4484 - acc: 0.8187 - val_loss: 0.3248 - val_acc: 0.8575
Epoch 2245/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4562 - acc: 0.8275 - val_loss: 0.3594 - val_acc: 0.8175
Epoch 2246/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4604 - acc: 0.8137 - val_loss: 0.3185 - val_acc: 0.7075
Epoch 2247/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4114 - acc: 0.7856 - val_loss: 0.3465 - val_acc: 0.8500
Epoch 2248/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.3904 - acc: 0.8244 - val_loss: 0.3179 - val_acc: 0.8575
Epoch 2249/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4200 - acc: 0.8300 - val_loss: 0.3162 - val_acc: 0.8575
Epoch 2250/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.5066 - acc: 0.8163 - val_loss: 0.3209 - val_acc: 0.8575
Epoch 2251/2500
1600/1600 [

Epoch 2303/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4332 - acc: 0.8156 - val_loss: 0.3322 - val_acc: 0.8450
Epoch 2304/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.4317 - acc: 0.8244 - val_loss: 0.3381 - val_acc: 0.8450
Epoch 2305/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.4178 - acc: 0.8294 - val_loss: 0.3301 - val_acc: 0.8550
Epoch 2306/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4220 - acc: 0.8013 - val_loss: 0.3298 - val_acc: 0.8575
Epoch 2307/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4347 - acc: 0.8256 - val_loss: 0.3477 - val_acc: 0.8525
Epoch 2308/2500
1600/1600 [==============================] - 7s 5ms/step - loss: 0.3754 - acc: 0.7756 - val_loss: 0.3282 - val_acc: 0.8575
Epoch 2309/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4613 - acc: 0.8050 - val_loss: 0.3338 - val_acc: 0.8575
Epoch 2310/2500
1600/1600 [

Epoch 2362/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4467 - acc: 0.8219 - val_loss: 0.3449 - val_acc: 0.8525
Epoch 2363/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4306 - acc: 0.8244 - val_loss: 0.3370 - val_acc: 0.8525
Epoch 2364/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4334 - acc: 0.8231 - val_loss: 0.3341 - val_acc: 0.8575
Epoch 2365/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4287 - acc: 0.8294 - val_loss: 0.3666 - val_acc: 0.8525
Epoch 2366/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4395 - acc: 0.8237 - val_loss: 0.3502 - val_acc: 0.8500
Epoch 2367/2500
1600/1600 [==============================] - 7s 5ms/step - loss: 0.4117 - acc: 0.8206 - val_loss: 0.3399 - val_acc: 0.8500
Epoch 2368/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.5109 - acc: 0.8250 - val_loss: 0.3796 - val_acc: 0.8550
Epoch 2369/2500
1600/1600 [

Epoch 2421/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.3857 - acc: 0.8250 - val_loss: 0.3752 - val_acc: 0.8475
Epoch 2422/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4387 - acc: 0.8187 - val_loss: 0.3466 - val_acc: 0.8475
Epoch 2423/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4476 - acc: 0.8231 - val_loss: 0.3439 - val_acc: 0.8425
Epoch 2424/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4358 - acc: 0.8156 - val_loss: 0.3733 - val_acc: 0.8450
Epoch 2425/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4260 - acc: 0.8187 - val_loss: 0.3657 - val_acc: 0.8475
Epoch 2426/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4499 - acc: 0.8250 - val_loss: 0.3662 - val_acc: 0.8350
Epoch 2427/2500
1600/1600 [==============================] - 6s 4ms/step - loss: 0.4841 - acc: 0.8225 - val_loss: 0.3516 - val_acc: 0.8450
Epoch 2428/2500
1600/1600 [

Epoch 2480/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4545 - acc: 0.8056 - val_loss: 0.3532 - val_acc: 0.7825
Epoch 2481/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4579 - acc: 0.7956 - val_loss: 0.3676 - val_acc: 0.8300
Epoch 2482/2500
1600/1600 [==============================] - 7s 5ms/step - loss: 0.5034 - acc: 0.7963 - val_loss: 0.3495 - val_acc: 0.8425
Epoch 2483/2500
1600/1600 [==============================] - 9s 6ms/step - loss: 0.4349 - acc: 0.8225 - val_loss: 0.3989 - val_acc: 0.8325
Epoch 2484/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4719 - acc: 0.8175 - val_loss: 0.3510 - val_acc: 0.8275
Epoch 2485/2500
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4398 - acc: 0.8038 - val_loss: 0.3479 - val_acc: 0.8300
Epoch 2486/2500
1600/1600 [==============================] - 8s 5ms/step - loss: 0.4448 - acc: 0.8144 - val_loss: 0.3344 - val_acc: 0.8325
Epoch 2487/2500
1600/1600 [

Wow, I did overtrain it above!

In [35]:
model.fit(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), 
          batch_size=1, epochs=10, verbose=1)


Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4517 - acc: 0.6600 - val_loss: 0.3404 - val_acc: 0.8500
Epoch 2/10
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4369 - acc: 0.8231 - val_loss: 0.3579 - val_acc: 0.8475
Epoch 3/10
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4487 - acc: 0.7850 - val_loss: 0.3549 - val_acc: 0.8450
Epoch 4/10
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4557 - acc: 0.8181 - val_loss: 0.3469 - val_acc: 0.8450
Epoch 5/10
1600/1600 [==============================] - 7s 5ms/step - loss: 0.4911 - acc: 0.8181 - val_loss: 0.3500 - val_acc: 0.8450
Epoch 6/10
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4828 - acc: 0.7956 - val_loss: 0.3679 - val_acc: 0.8400
Epoch 7/10
1600/1600 [==============================] - 7s 4ms/step - loss: 0.4697 - acc: 0.8125 - val_loss: 0.3455 - val_acc: 0.8500
Epoch 8/10
1600

In [36]:
validation_inputs.shape

(400, 5, 10)

In [37]:
len(validation_labels)

400

In [38]:
scores = model.evaluate(validation_inputs, validation_labels, batch_size=1,verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.75%


In [39]:
model.save("Guillaume_model.h5")